In [1]:
from model import Model

c:\Users\a.almasri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = Model(model_name="all-MiniLM-L6-v2")

In [3]:
m.embed(sentences=["Hello world"])

TypeError: BertModel.forward() got an unexpected keyword argument 'padding'

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']
sentences = ["Hello world"]
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')




In [9]:
from typing import List


In [10]:
def encode(sentences: List[str] = None, normalzied: bool = True) -> torch.Tensor:

    if sentences is not None:
            
        # Tokenize sentences
        encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

        if normalzied:
            # Normalize embeddings
            sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

        return sentence_embeddings

In [11]:
print(encode(["Hello world"]))

tensor([[-3.4477e-02,  3.1023e-02,  6.7349e-03,  2.6109e-02, -3.9362e-02,
         -1.6030e-01,  6.6924e-02, -6.4415e-03, -4.7451e-02,  1.4759e-02,
          7.0875e-02,  5.5528e-02,  1.9193e-02, -2.6251e-02, -1.0110e-02,
         -2.6941e-02,  2.2308e-02, -2.2227e-02, -1.4969e-01, -1.7493e-02,
          7.6763e-03,  5.4352e-02,  3.2544e-03,  3.1726e-02, -8.4621e-02,
         -2.9406e-02,  5.1596e-02,  4.8124e-02, -3.3148e-03, -5.8279e-02,
          4.1969e-02,  2.2211e-02,  1.2819e-01, -2.2339e-02, -1.1656e-02,
          6.2928e-02, -3.2876e-02, -9.1226e-02, -3.1175e-02,  5.2700e-02,
          4.7035e-02, -8.4203e-02, -3.0056e-02, -2.0745e-02,  9.5178e-03,
         -3.7218e-03,  7.3434e-03,  3.9324e-02,  9.3274e-02, -3.7887e-03,
         -5.2742e-02, -5.8058e-02, -6.8644e-03,  5.2832e-03,  8.2893e-02,
          1.9363e-02,  6.2845e-03, -1.0331e-02,  9.0324e-03, -3.7684e-02,
         -4.5206e-02,  2.4016e-02, -6.9442e-03,  1.3492e-02,  1.0006e-01,
         -7.1684e-02, -2.1695e-02,  3.

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from typing import List
from utils import mean_pooling


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class EmbeddingModel:
    """
    A class to wrap the embedding model to be used

    Attributes:

    """

    allowed_models = ["all-mpnet-base-v2",
                      "all-MiniLM-L6-v2"]

    def __init__(self, model_name: str = "all-mpnet-base-v2") -> None:
        """ 
        """
        if model_name in self.allowed_models:
            # Load model from HuggingFace Hub
            self.model = AutoTokenizer.from_pretrained(
                "sentence-transformers/all-MiniLM-L6-v2"
            )
            self.tokenizer = AutoModel.from_pretrained(
                "sentence-transformers/all-MiniLM-L6-v2"
            )
        else:
            raise Exception("Model not supported!")

    def emb(self, sentences: List[str] = None, normalzied: bool = True) -> torch.Tensor:
        """
        """
        if sentences is not None:
            # Tokenize sentences
            encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

            # Compute token embeddings
            with torch.no_grad():
                model_output = self.model(**encoded_input)

            # Perform pooling
            sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

            if normalzied:
                # Normalize embeddings
                sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

            return sentence_embeddings           

In [ ]:
m = EmbeddingModel(model_name="all-MiniLM-L6-v2")
m.emb(["Hello world"])

In [20]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from typing import List
from utils import mean_pooling



def embed(sentences, normalzied):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')
   
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input) 

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    if normalzied:
        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings



In [22]:
m.embed(["Hello world", "I am here"], normalzied=False)

tensor([[ 0.0719,  0.0367, -0.0124,  ...,  0.0974, -0.0844, -0.0289],
        [ 0.0844,  0.0285,  0.0038,  ...,  0.2837, -0.0755, -0.0455]])

In [1]:
from model import EmbeddingModel
from utils import mean_pooling

In [2]:
m = EmbeddingModel()

In [8]:
l = m.embed(["Hello world", "I am here"], normalzied=False, return_tensors=True)

### Session Class

In [1]:
from model import EmbeddingModel

In [30]:
queries = ["Hello world", "I am here", "I am here", "I am here"]

In [3]:
m = EmbeddingModel("all-mpnet-base-v2")

In [38]:
from typing import List, Union
import numpy as np
import torch
import pandas as pd
import numpy as np
import numpy as np


class QuerySession:
    """
    """    

    def __init__(self, queries: List[str] = None, normalize_embeddings: bool = False, embedding_model: str = None) -> None:
        """
        """
        if queries is not None:
            self.queries: List[str] = queries[:]
            self.session_size = len(queries)
            self.embedding_model = embedding_model
            self.normalized_session = normalize_embeddings
            self.embeddings = self._embed_queries(queries=queries, embedding_model=self.embedding_model, normalize_embeddings=self.normalized_session)
            self.global_coherence_score = self.compute_global_coherence(context_window=1)
            self.context_window_size = 1
                       
            
        
        

    def _embed_queries(self, queries: List[str], embedding_model: str = None, normalize_embeddings: bool = False) -> np.ndarray:
        """
        """
        if embedding_model is not None:
            return embedding_model.embed(sentences=queries, normalize_embeddings=normalize_embeddings)

    def add_query(self, query) -> None:
        """
        """
        self.queries.append(query)
        self.session_size += 1
        query_embedding = self.embedding_model.embed(sentences=[query], normalize_embeddings=self.normalized_session)
        print(query_embedding.shape)
        self.embeddings = np.concatenate([self.embeddings , query_embedding])

    def _compute_context_vector(self, position, context_window) -> np.ndarray:
        """
        """
        if context_window >= self.session_size:
            raise ValueError("Context window is too big")

        if position < context_window:
            context_vector = np.sum(self.embeddings[:position], axis=0)
            context_vector = context_vector / np.linalg.norm(context_vector)
            
        
        else:
            context_vector = np.sum(self.embeddings[position-context_window:position], axis=0)
            context_vector = context_vector / np.linalg.norm(context_vector)

        return context_vector        
            
      
        

    def _compute_local_coherence(self, current_vector: np.ndarray = None, neighbor_vector: np.ndarray = None) -> float:
        """
        """
        if current_vector is not None and neighbor_vector is not None:
            return np.dot(current_vector, neighbor_vector) / (np.linalg.norm(current_vector) * np.linalg.norm(neighbor_vector))
    
        
    def compute_global_coherence(self, context_window) -> float:
        """
        """
        if self.session_size > 1:

            coherence_score = 0.0

            for i, embedding in enumerate(self.embeddings):
                if i == 0:
                    continue
                context_vector = self._compute_context_vector(position=i, context_window=context_window)
                coherence_score += self._compute_local_coherence(current_vector=embedding, neighbor_vector=context_vector)
            
        return coherence_score / (self.session_size - 1)

        
        

        
        
    




In [39]:
s = QuerySession(queries=queries, embedding_model=m)

In [40]:
s.queries

['Hello world', 'I am here', 'I am here', 'I am here']

In [42]:
s.global_coherence_score

0.7591788669427236

In [28]:
s._compute_global_coherence(context_window=1)

In [43]:
e = embed(queries, normalzied=False)

In [44]:
coherence_score = 0.0

for i, embedding in enumerate(e):
    if i == 0:
        continue
    coherence_score += np.dot(embedding, e[i-1]) / (np.linalg.norm(embedding) * np.linalg.norm(e[i-1]))

coherence_score / (len(e) - 1)

0.7591788570086161